In [1]:
import folium, geopandas as gpd, pandas as pd, fiona, requests, json, numpy as np
from shapely.geometry import Point

In [2]:
uscities = gpd.read_file('https://gist.githubusercontent.com/Vudude/cee778c78694fe4183aec99693e104b1/raw/6108e7f372813f261c29ae085659ec66118285ac/cities.geo.json')
uscities['population']=pd.to_numeric(uscities['population'], errors='coerce')
uscities = uscities.sort_values(by=['population'], ascending=[False]).iloc[:200].dropna()
uscities['ranking']=pd.Series([num+1 for num in uscities.index])
uscities.head()

,city,growth_10-13,population,state,geometry,ranking
0,New York,4.8%,8405837,New York,POINT (-74.0059413 40.7127837),1
1,Los Angeles,4.8%,3884307,California,POINT (-118.2436849 34.0522342),2
2,Chicago,-6.1%,2718782,Illinois,POINT (-87.6297982 41.8781136),3
3,Houston,11.0%,2195914,Texas,POINT (-95.3698028 29.7604267),4
4,Philadelphia,2.6%,1553165,Pennsylvania,POINT (-75.1652215 39.9525839),5


In [3]:
states = gpd.read_file("http://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_500k.json")
states['geometry']=states.geometry.simplify(tolerance=0.01)
states.head()

,GEO_ID,STATE,NAME,LSAD,CENSUSAREA,geometry
0,0400000US23,23,Maine,,30842.923,"(POLYGON ((-67.619761 44.519754, -67.582113000..."
1,0400000US25,25,Massachusetts,,7800.058,"(POLYGON ((-70.832044 41.606504, -70.823735 41..."
2,0400000US26,26,Michigan,,56538.901,"(POLYGON ((-88.684434 48.115785, -88.656915 48..."
3,0400000US30,30,Montana,,145545.801,"POLYGON ((-104.057698 44.997431, -111.055199 4..."
4,0400000US32,32,Nevada,,109781.180,"POLYGON ((-114.0506 37.000396, -114.046838 36...."


In [5]:
centroid=uscities.geometry.centroid

m = folium.Map(location=[40, -97], 
               zoom_start=4,tiles='mapboxbright')

folium.GeoJson(data=states,
               name='US States',smooth_factor=2,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':2},
                tooltip=folium.GeoJsonTooltip(fields=['NAME'],
                                              labels=False,
                                              sticky=False),
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'}
              ).add_to(m)


def stylefunc(feat):
    obj={'weight':1}
    props = feat['properties']
    growth=float(props['growth_10-13'].replace('%',''))
    obj['fillColor']='green' if growth > 0 else 'red'
    obj['color']='black'
    obj['fillOpacity']=0.5
    obj['radius']=(np.log2(props['population'])**4)/3
    return obj

from folium.plugins import MarkerCluster

folium.GeoJson(uscities, 
               name='Top US Cities by Population',
               marker=folium.Circle(location=None, fill=True, radius=False, fill_opacity=0.5), 
               style_function=stylefunc,
               tooltip=folium.GeoJsonTooltip(fields=['city','state','growth_10-13','population','ranking'],
                                             aliases=['<div style="background-color: lightyellow; color: black; padding: 3px; border: 2px solid black; border-radius: 3px;">'+item+'</div>' for item in ['City','State','Growth, 2010-2013', 'Population','Ranking']],
                                             style="font-family: san serif;",
                                            localize=True),
               highlight_function=lambda x: {"weight":3,'fillOpacity':0.95,'zoom':8}
              ).add_to(m)
folium.LayerControl(autoZIndex=True).add_to(m)

m